# MetraTrader5

If you have this error importing MetaTrader5 package:

```python
ImportError: numpy.core.multiarray failed to import
```

You have to upgrade numpy <code>pip install numpy --upgrade</code>. Run Anaconda Prompt as **Administrator** (for Windows)

[MetaTrader5 documentation](https://www.mql5.com/en/docs/integration/python_metatrader5)


**Jupyter themes**
```python
# List all the jupyter themes availables
!jt -l

# Restore to original theme
!jt -r

# Set specific theme
!jt -t solarizedl
```

## Initialize

In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz

In [2]:
# https://towardsdatascience.com/8-commonly-used-pandas-display-options-you-should-know-a832365efa95
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# Show data MetaTrader5 package
print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


In [4]:
# Establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

In [5]:
# Display data on MetaTrader 5 version
print(mt5.version())

(500, 3280, '29 Apr 2022')


## Info

### Account

In [6]:
# Connect to the trade account specifying a password and a server. Timeout= 30 seconds
mt5_log = {'user': 48106721, 'password': "AVtb*USu", 'server': "HFMarketsGlobal-Demo", 'timeout': 30000}

# Log in
authorized = mt5.login(mt5_log['user'], password=mt5_log['password'], server=mt5_log['server'], timeout=mt5_log['timeout']) 

if authorized:
    # Get trading account data
    account_info_dict = mt5.account_info()._asdict()
    
    # Convert to DataFrame
    account_info = pd.DataFrame(list(account_info_dict.items()), columns=['Property','Value'])

    # Set "Property" column as index
    account_info.set_index('Property', inplace=True)
else:
    print("failed to connect at account #{}, error code: {}".format(mt5_log['user'], mt5.last_error()))

In [7]:
mt5_log['user']

48106721

In [8]:
account_info.index

Index(['login', 'trade_mode', 'leverage', 'limit_orders', 'margin_so_mode',
       'trade_allowed', 'trade_expert', 'margin_mode', 'currency_digits',
       'fifo_close', 'balance', 'credit', 'profit', 'equity', 'margin',
       'margin_free', 'margin_level', 'margin_so_call', 'margin_so_so',
       'margin_initial', 'margin_maintenance', 'assets', 'liabilities',
       'commission_blocked', 'name', 'server', 'currency', 'company'],
      dtype='object', name='Property')

In [9]:
account_info.loc['balance', 'Value'], account_info.loc['equity', 'Value']

(100112.99, 83488.4)

In [10]:
account_info.loc['leverage', 'Value'], account_info.loc['margin', 'Value'], account_info.loc['margin_free', 'Value']

(500, 3523.6, 79964.8)

### Terminal

In [11]:
# Get terminal settings and status
terminal_info_raw = mt5.terminal_info()

if terminal_info_raw != None:
    # Convert to dictionary
    terminal_info_dict = mt5.terminal_info()._asdict()

   # Convert the dictionary into DataFrame
    terminal_info=pd.DataFrame(list(terminal_info_dict.items()), columns=['Property', 'Value'])

    # Set "Property" column as index
    terminal_info.set_index('Property', inplace=True)

In [12]:
terminal_info.index

Index(['community_account', 'community_connection', 'connected',
       'dlls_allowed', 'trade_allowed', 'tradeapi_disabled', 'email_enabled',
       'ftp_enabled', 'notifications_enabled', 'mqid', 'build', 'maxbars',
       'codepage', 'ping_last', 'community_balance', 'retransmission',
       'company', 'name', 'language', 'path', 'data_path', 'commondata_path'],
      dtype='object', name='Property')

In [13]:
terminal_info.loc['connected', 'Value']

True

### Symbols

#### Number of symbols

In [14]:
# get the number of financial instruments
total_symbols = mt5.symbols_total()

print("Total symbols =", total_symbols) if total_symbols > 0 else print("Symbols not found")

Total symbols = 1050


#### Symbols names

In [15]:
# Get all symbols
symbols = mt5.symbols_get()

print('Symbols: ', len(symbols))

# Display the first symbols
count=1

for item in range(5):
    print("{}. {}".format(item+1, symbols[item].name))

Symbols:  1050
1. EURUSD
2. GBPUSD
3. USDCHF
4. USDJPY
5. USDCAD


In [16]:
# Get symbols containing "00" in their names
zero_symbols = mt5.symbols_get(group="*00*")
print('Lenght "00" symbols: ', len(zero_symbols))

for item in zero_symbols:
    print(item.name)

Lenght "00" symbols:  7
UK100.F
US100.F
US500.F
AUS200
UK100
USA500
USA100


In [17]:
# Get symbols containing "00" in their names
f_symbols = mt5.symbols_get(group="*.F*")
print('Lenght .F symbols: ', len(f_symbols))

for item in f_symbols:
    print(item.name)

Lenght .F symbols:  14
EU50.F
FRA40.F
JP225.F
N25.F
SUI20.F
UK100.F
US100.F
US30.F
US500.F
EUBUND.F
UKGILT.F
US10YR.F
VIX.F
GER40.F


#### Symbols info

In [18]:
# Display of the USA500 symbol in MarketWatch
selected = mt5.symbol_select("USA500", True)

if not selected:
    print("Failed to select USA500")
    mt5.shutdown()
    quit()

In [19]:
# Get data on the specified financial instrument.
# USA500 symbol properties
symbol_info = mt5.symbol_info("USA500")

if symbol_info != None:
    print("USA500: spread =", symbol_info.spread, "  digits =", symbol_info.digits)
    
    # Symbol properties as a dataframe
    symbol_info_dict = mt5.symbol_info("USA500")._asdict()
    df_symbol = pd.DataFrame(list(symbol_info_dict.items()), columns=['Property','Value'])

    # Set "Property" column as index
    df_symbol.set_index('Property', inplace=True)

USA500: spread = 34   digits = 2


In [20]:
symbol_info

SymbolInfo(custom=False, chart_mode=0, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1651881536, digits=2, spread=34, spread_float=True, ticks_bookdepth=0, trade_calc_mode=2, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=20, trade_freeze_level=0, trade_exemode=2, swap_mode=3, swap_rollover3days=5, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=1, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=4118.81, bidhigh=4159.2, bidlow=4068.7, ask=4119.15, askhigh=4159.52, asklow=4069.02, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=0.01, trade_tick_value=0.1, trade_tick_value_profit=0.1, trade_tick_value_loss=0.1, trade_tick_size=0.01, trade_contract_size=1.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=0.1, volume_max=60.0, volume_step=0.01,

In [21]:
df_symbol.index

Index(['custom', 'chart_mode', 'select', 'visible', 'session_deals',
       'session_buy_orders', 'session_sell_orders', 'volume', 'volumehigh',
       'volumelow', 'time', 'digits', 'spread', 'spread_float',
       'ticks_bookdepth', 'trade_calc_mode', 'trade_mode', 'start_time',
       'expiration_time', 'trade_stops_level', 'trade_freeze_level',
       'trade_exemode', 'swap_mode', 'swap_rollover3days',
       'margin_hedged_use_leg', 'expiration_mode', 'filling_mode',
       'order_mode', 'order_gtc_mode', 'option_mode', 'option_right', 'bid',
       'bidhigh', 'bidlow', 'ask', 'askhigh', 'asklow', 'last', 'lasthigh',
       'lastlow', 'volume_real', 'volumehigh_real', 'volumelow_real',
       'option_strike', 'point', 'trade_tick_value', 'trade_tick_value_profit',
       'trade_tick_value_loss', 'trade_tick_size', 'trade_contract_size',
       'trade_accrued_interest', 'trade_face_value', 'trade_liquidity_rate',
       'volume_min', 'volume_max', 'volume_step', 'volume_limit', 'sw

In [22]:
df_symbol.loc['description', 'Value']

'S&P 500 / Mini CFD'

In [23]:
df_symbol.loc['time', 'Value']

1651881536

In [24]:
df_symbol.loc['bid', 'Value'], df_symbol.loc['ask', 'Value'], df_symbol.loc['last', 'Value']

(4118.81, 4119.15, 0.0)

#### Symbols info tick

In [25]:
# Get last tick of the specified financial instrument

# US500.F
last_tick_500 = mt5.symbol_info_tick("US500.F")

# Tick field values in the form of a list
symbol_info_tick_dict = mt5.symbol_info_tick("US500.F")._asdict()

df_symbol_info_tick = pd.DataFrame(list(symbol_info_tick_dict.items()), columns=['Property','Value'])

# Set "Property" column as index
df_symbol_info_tick.set_index('Property', inplace=True)

In [26]:
df_symbol_info_tick

,Value
Property,
time,1651881539.00
bid,4112.44
ask,4113.27
last,0.00
volume,0.00
time_msc,1651881539737.00
flags,6.00
volume_real,0.00


In [27]:
# # Initialize MT5

# if not mt5.initialize(login=48106721, server="HFMarketsGlobal-Demo", password="AVtb*USu", timeout=30000):
#     # Timeout= 30 seconds (30000 miliseconds)
#     print("initialize() failed, error code =",mt5.last_error())
#     quit()
    
# mt5.terminal_info()    

In [28]:
# # finalizamos la conexión con el terminal MetaTrader 5
# mt5.shutdown()

## Get data

<b>Market hours (RTH):</b>
- 9:30 - 16:00 ET
- 13:30 - 20:00 UTC


<b>Set datetime in <code>copy_rates()</code> functions:</b>
- Local time: 12:30 to 18:30.
- UTC: 16:30 to 22:30.
- ET: 11:30 to 18:00.

Get bars starting from the specified date backwards.

In [29]:
# Set time zone
timezone_utc = pytz.timezone("Etc/UTC")
timezone_et = pytz.timezone("US/Eastern")

# Create 'datetime' object 
# Returns in UTC time zone to avoid the implementation of a local time zone offset
# utc_time = datetime(2022, 4, 29, 19, 00, 0, tzinfo=timezone_utc)

# Returns local time server data
from_date = datetime(2022, 4, 29, 18, 00, 0, tzinfo=timezone_utc)

# Get 10 USA500 30 mins bars starting from 04.29.2020 18:00 in UTC time zone. Roll back
rates = mt5.copy_rates_from("US500.F", mt5.TIMEFRAME_M30, from_date, 5)

data_df = pd.DataFrame(rates)

# convert time in seconds into the datetime format
data_df['time'] = pd.to_datetime(data_df['time'], unit='s')

In [30]:
data_df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-04-29 16:00:00,4240.94,4243.70,4229.70,4242.07,2193,83,0
1,2022-04-29 16:30:00,4242.94,4264.94,4231.94,4263.20,7632,83,0
2,2022-04-29 17:00:00,4263.32,4264.70,4215.94,4217.70,6360,83,0
3,2022-04-29 17:30:00,4217.94,4220.20,4205.70,4213.70,4367,83,0
4,2022-04-29 18:00:00,4212.94,4218.20,4202.44,4215.44,4516,83,0


Get bars starting from the specified index backwards.

In [31]:
# Get the last 10 US500.F bars from the current date
# copy_rates_from_pos(symbol, timeframe, start_pos, count)
rates = mt5.copy_rates_from_pos("US500.F", mt5.TIMEFRAME_M30, 0, 10)

data_df = pd.DataFrame(rates)

# convert time in seconds into the datetime format
data_df['time'] = pd.to_datetime(data_df['time'], unit='s')

In [32]:
data_df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-05-06 19:00:00,4103.20,4135.44,4099.70,4123.20,7579,83,0
1,2022-05-06 19:30:00,4122.70,4143.94,4112.94,4142.44,8137,83,0
2,2022-05-06 20:00:00,4143.20,4147.20,4114.44,4124.94,6811,83,0
3,2022-05-06 20:30:00,4125.20,4133.82,4102.20,4105.44,7640,83,0
4,2022-05-06 21:00:00,4105.70,4111.70,4087.57,4091.70,6479,83,0
5,2022-05-06 21:30:00,4091.44,4110.44,4082.44,4093.20,6777,83,0
6,2022-05-06 22:00:00,4093.44,4098.44,4077.94,4090.44,6418,83,0
7,2022-05-06 22:30:00,4090.20,4134.44,4089.94,4118.94,8060,83,0
8,2022-05-06 23:00:00,4119.70,4119.70,4102.70,4107.70,1960,83,0
9,2022-05-06 23:30:00,4107.57,4114.20,4106.32,4112.44,677,83,0


Get bars in the specified date range.

In [33]:
# Set time zone
timezone_utc = pytz.timezone("Etc/UTC")
timezone_et = pytz.timezone("US/Eastern")

# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
from_date = datetime(2022, 4, 29, 11, 30, 0, tzinfo=timezone_et)
to_date = datetime(2022, 4, 29, 18, 0, 0, tzinfo=timezone_et)

# from_date = datetime(2022, 4, 29, 16, 30, 0, tzinfo=timezone_utc)
# to_date = datetime(2022, 4, 29, 22, 30, 0, tzinfo=timezone_utc)

# # Local time to get RTH data
# from_date = datetime(2022, 4, 29, 12, 30, 0)
# to_date = datetime(2022, 4, 29, 18, 30, 0)

# Get bars from the specified dates
rates = mt5.copy_rates_range("US500.F", mt5.TIMEFRAME_M30, from_date, to_date)

data_df = pd.DataFrame(rates)

# convert time in seconds into the datetime format
data_df['time'] = pd.to_datetime(data_df['time'], unit='s')

In [34]:
data_df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-04-29 16:30:00,4242.94,4264.94,4231.94,4263.20,7632,83,0
1,2022-04-29 17:00:00,4263.32,4264.70,4215.94,4217.70,6360,83,0
2,2022-04-29 17:30:00,4217.94,4220.20,4205.70,4213.70,4367,83,0
3,2022-04-29 18:00:00,4212.94,4218.20,4202.44,4215.44,4516,83,0
4,2022-04-29 18:30:00,4215.20,4227.44,4210.44,4210.94,4361,83,0
5,2022-04-29 19:00:00,4210.82,4211.94,4192.70,4194.44,2837,83,0
6,2022-04-29 19:30:00,4194.20,4195.94,4178.20,4182.94,2665,83,0
7,2022-04-29 20:00:00,4182.70,4200.70,4181.20,4192.70,3085,83,0
8,2022-04-29 20:30:00,4192.44,4203.94,4181.94,4182.70,2818,83,0
9,2022-04-29 21:00:00,4182.94,4186.70,4171.44,4173.20,3194,83,0


I am not going to test <b>Ticks</b> (data every second)

## Orders

Get the number of active orders.

In [35]:
# check the presence of active orders
orders = mt5.orders_total()

print("Total orders=", orders) if orders > 0 else print("Orders not found")

Orders not found


Get active orders with the ability to filter by symbol or ticket. The function <code>orders_get()</code> has the input "ticket" (nique number assigned to each order)

In [36]:
# Display data on active orders on US500.F
orders = mt5.orders_get(symbol="US500.F")

if orders is None:
    print("No orders on US500.F, error code={}".format(mt5.last_error()))
    
else:
    print("Total orders on US500.F:", len(orders))
    
    # Display all active orders
    for item in orders:
        print(item)

Total orders on US500.F: 0


Return margin in the account currency to perform a specified trading operation.

The function allows estimating the margin necessary for a specified order type on the current account and in the current market environment without considering the current pending orders and open positions.

A standard lot contains <b>100,000</b> units of the base currency. 

In [37]:
# Get account currency
account_currency = mt5.account_info().currency
print("Account currency:",account_currency)
 
# Arrange the symbol list
symbols = ("EURUSD", "GBPUSD", "USDJPY", "USDCHF", "EURJPY", "GBPJPY", "US500.F")
print("Symbols to check margin:", symbols)

Account currency: USD
Symbols to check margin: ('EURUSD', 'GBPUSD', 'USDJPY', 'USDCHF', 'EURJPY', 'GBPJPY', 'US500.F')


In [38]:
action = mt5.ORDER_TYPE_BUY
lot = 0.1

for symbol in symbols:

    # Get symbol info
    symbol_info = mt5.symbol_info(symbol)

    if symbol_info is None:
        print(symbol, "not found, skipped")
        continue
    
    # Check if symbol is in MarketWatch
    if not symbol_info.visible:
        print(symbol, "is not visible, trying to switch on")

        if not mt5.symbol_select(symbol, True):
            print("symbol_select({}}) failed, skipped",symbol)
            continue
    
    # Ask price
    ask = mt5.symbol_info_tick(symbol).ask

    # Margin to perform a specified operation
    margin = mt5.order_calc_margin(action, symbol, lot, ask)

    if margin != None:
        print("{} buy {} lot. Margin: {} {}. Ask price {}".format(symbol, lot, margin, account_currency, ask))

    else:
        print("order_calc_margin failed: , error code =", mt5.last_error())
 

EURUSD buy 0.1 lot. Margin: 21.1 USD. Ask price 1.05515
GBPUSD buy 0.1 lot. Margin: 24.7 USD. Ask price 1.23498
USDJPY is not visible, trying to switch on
USDJPY buy 0.1 lot. Margin: 20.0 USD. Ask price 130.573
USDCHF is not visible, trying to switch on
USDCHF buy 0.1 lot. Margin: 20.0 USD. Ask price 0.98876
EURJPY is not visible, trying to switch on
order_calc_margin failed: , error code = (-2, 'Terminal: Invalid params')
GBPJPY is not visible, trying to switch on
order_calc_margin failed: , error code = (-2, 'Terminal: Invalid params')
US500.F buy 0.1 lot. Margin: 2.06 USD. Ask price 4113.27


Send new order

In [39]:
# Prepare the order request structure
symbol = "US500.F"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, ": Not found, can not call order_check()")
    mt5.shutdown()
    quit()

In [40]:
# If the symbol is unavailable in MarketWatch, add it
if not symbol_info.visible:
    print(symbol, ": Is not visible, trying to switch on")
    
    if not mt5.symbol_select(symbol,True):
        print("symbol_select({}}) failed, exit",symbol)
        mt5.shutdown()
        quit()

In [41]:
lot = 10.0
price = mt5.symbol_info_tick(symbol).ask
deviation = 1000
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": price,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,
}
 

In [42]:
# Send a trading request
result = mt5.order_send(request)

In [43]:
result

OrderSendResult(retcode=10027, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='AutoTrading disabled by client', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='US500.F', volume=10.0, price=4113.27, stoplimit=0.0, sl=0.0, tp=0.0, deviation=1000, type=1, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [44]:
mt5.TRADE_RETCODE_DONE

10009

In [45]:
# Send a trading request
result = mt5.order_send(request)

# check the execution result
print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation))

if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
    print("shutdown() and quit")
    mt5.shutdown()
    quit()

1. order_send(): by US500.F 10.0 lots at 4113.27 with deviation=1000 points
2. order_send failed, retcode=10027
   retcode=10027
   deal=0
   order=0
   volume=0.0
   price=0.0
   bid=0.0
   ask=0.0
   comment=AutoTrading disabled by client
   request_id=0
   retcode_external=0
   request=TradeRequest(action=1, magic=234000, order=0, symbol='US500.F', volume=10.0, price=4113.27, stoplimit=0.0, sl=0.0, tp=0.0, deviation=1000, type=1, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0)
       traderequest: action=1
       traderequest: magic=234000
       traderequest: order=0
       traderequest: symbol=US500.F
       traderequest: volume=10.0
       traderequest: price=4113.27
       traderequest: stoplimit=0.0
       traderequest: sl=0.0
       traderequest: tp=0.0
       traderequest: deviation=1000
       traderequest: type=1
       traderequest: type_filling=0
       traderequest: type_time=0
       traderequest: expiration=0
       t

: 